<a href="https://colab.research.google.com/github/newb-dev-1008/Human-PokeDex/blob/colab/Human_PokeDex.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install --upgrade imutils

Requirement already up-to-date: imutils in /usr/local/lib/python3.7/dist-packages (0.5.4)


In [2]:
!pip install opencv-contrib-python

# **Extract embeddings from face dataset**

In [4]:
# Importing all necessary packages

from imutils import paths
import numpy as np
import argparse
import imutils
import pickle
import time
import cv2
import os